In [6]:
import os
import pandas as pd

In [7]:
input_folder = "/content/drive/MyDrive/Company_Raw_Data/Chemicals"
output_folder = "/content/drive/MyDrive/Cleaned Data/Chemicals"
os.makedirs(output_folder, exist_ok=True)

In [8]:
# Sheet names to extract
sheet_names = ["Balance Sheet", "Cash Flow", "Profit & Loss", "Quarters"]

In [9]:
def clean_df(df):
    # Clean column names
    df.columns = (
        df.columns
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'[^\w\s]', '', regex=True)
        .str.strip()
    )

    # Clean index
    df.index = (
        df.index.astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

    # Clean first column (typically labels)
    if df.shape[1] > 1:
        df.iloc[:, 0] = (
            df.iloc[:, 0].astype(str)
            .str.replace(r'\+', '', regex=True)
            .str.replace(r'%', '', regex=True)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

    # Remove all % signs from entire DataFrame
    df = df.replace(r'%', '', regex=True)

    # Convert columns (except first) to numeric
    for col in df.columns[1:]:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')


    # Convert row index into column temporarily
    df.reset_index(inplace=True)

    # Rename first column (previous row labels) to 'Micro_Data'
    df.rename(columns={'Unnamed 0': 'Financial_Metric'}, inplace=True)

    # Drop columns with more than 50% missing values
    df = df.dropna(thresh=0.5 * len(df), axis=1)

    # Restore the original index (first column becomes new index)
    df.set_index(df.columns[0], inplace=True)

    return df


In [10]:
# Process and save one Excel file
def process_excel_file(file_path):
    company_name = os.path.splitext(os.path.basename(file_path))[0]
    print(f"\n📂 Processing: {company_name}")

    try:
        xls = pd.ExcelFile(file_path)
        for sheet in sheet_names:
            if sheet in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet)
                df = clean_df(df)

                # Save to CSV
                sheet_clean_name = sheet.replace(' ', '_')
                output_file = os.path.join(output_folder, f"{company_name}_{sheet_clean_name}.csv")
                df.to_csv(output_file, index=False)
                print(f"✅ Saved: {output_file}")

                # Display first 10 rows
                print(f"\n📄 Cleaned Sheet: {company_name} - {sheet}")
                print(df.head(10).to_string(index=False))  # Pretty print

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")


In [11]:
# Loop through Excel files
for filename in os.listdir(input_folder):
    if filename.endswith(('.xlsx', '.xls')) and not filename.startswith('~$'):
        file_path = os.path.join(input_folder, filename)
        process_excel_file(file_path)


📂 Processing: UPL
✅ Saved: /content/drive/MyDrive/Cleaned Data/Chemicals/UPL_Balance_Sheet.csv

📄 Cleaned Sheet: UPL - Balance Sheet
 Financial_Metric  Mar 2014  Mar 2015  Mar 2016  Mar 2017  Mar 2018  Mar 2019  Mar 2020  Mar 2021  Mar 2022  Mar 2023  Mar 2024  Mar 2025
   Equity Capital        86        86        86       101       102       102       153       153       153       150       150       159
         Reserves      5162      5775      5803      7214      9067     14613     19129     20734     24508     26708     24657     29054
       Borrowings      3349      3281      5258      6443      6638     29139     29388     24519     26746     23939     29754     25099
Other Liabilities      4162      5143      5722      6454      7148     18581     19758     23359     29196     35318     30579     33690
Total Liabilities     12759     14284     16869     20212     22955     62435     68428     68765     80603     86115     85140     88002
     Fixed Assets      3821      4031 